In [12]:

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import networkx as nx
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

embeddings_model_name="all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name, model_kwargs={'device' : 'cpu'})
vectorstore2 = Chroma(persist_directory = "./TEST_chat/chroma", embedding_function=embeddings)

In [60]:
def auteurs_links(mult_items):

    aut_dic = {}

    for url in mult_items:
        for aut in mult_items[url]["auteurs"]:
            if aut == "N" or aut == "A":
                pass
            else:
                if aut not in aut_dic.keys():
                    aut_dic[aut] = {}
                    
    for aut in aut_dic.keys():
        for url in mult_items:
            if aut in mult_items[url]["auteurs"]:
                # aut_dic[aut]["value"] += 1
                for a in mult_items[url]["auteurs"]:
                    if a != aut and a not in aut_dic[aut].keys():
                        aut_dic[aut][a] = {"weight" : 1}
                    elif a != aut and a in aut_dic[aut].keys():
                        aut_dic[aut][a]["weight"] += 1

    g = nx.Graph(aut_dic)
    d = nx.to_pandas_adjacency(g)
    #d.values[[np.arange(len(d))]*2] = np.nan
    d = d.stack().reset_index()
    d.columns = ["source", "target", "values"]
    d = d[d["values"] > 0]
    d.to_csv("./pages/data/auteurs_link.csv")

def tag_links(mult_items):

    tag_dic = {}


    for url in mult_items:
        # print(url, len(mult_items[url]["tags"]))
        for tag in mult_items[url]["tags"]:
            if tag == "N" or tag == "A" or tag == '\n   \tThéo BATTAGLIA\n':
                pass
            else:
                if tag not in tag_dic.keys():
                    tag_dic[tag] = {}

    for tag in tag_dic.keys():
        for url in mult_items:
            if tag in mult_items[url]["tags"]:
                for t in mult_items[url]["tags"]:
                    if t in tag_dic.keys():
                        if t != tag and t not in tag_dic[tag].keys():
                            tag_dic[tag][t] = {"weight" : 1}
                        elif t != tag and t in tag_dic[tag].keys():
                            tag_dic[tag][t]["weight"] += 1

    g = nx.Graph(tag_dic)
    d = nx.to_pandas_adjacency(g)
    #d.values[[np.arange(len(d))]*2] = np.nan
    d = d.stack().reset_index()
    d.columns = ["source", "target", "values"]
    d = d[d["values"] > 0]
    d.to_csv("./pages/data/tags_link.csv")

In [61]:
import Lib.DATA2NET as net
mult_items = net.mult_items
tag_links(mult_items)
auteurs_links(mult_items)